# GPU4PySCF on Google Colab - 完全自動メモリ管理

このノートブックは、Google ColabからローカルのRTX 5070 Ti環境に接続し、GPU加速されたDFT計算を実行します。

**重要**: このノートブックは自動メモリクリーンアップ機能を使用しているため、キャッシュやメモリ管理を一切意識する必要がありません。

## ステップ1: ローカルランタイムへの接続

### WSL2側での準備
```bash
cd /path/to/computational-research-rtx50
./scripts/start-colab.sh
```

### Colab側での接続
1. 右上のメニューから「接続」→「ローカルランタイムに接続」を選択
2. WSL2で表示されたURL（例: `http://localhost:8888/?token=...`）を入力
3. 「接続」をクリック

接続できたら、次のセルを実行してください。

In [ ]:
# 接続確認
import sys
print(f"Python: {sys.version}")
print(f"Working directory: {sys.path[0]}")

# GPU確認
import cupy as cp
device = cp.cuda.Device()
print(f"\nGPU Device: {cp.cuda.runtime.getDeviceProperties(device.id)['name'].decode()}")

mem_info = cp.cuda.runtime.memGetInfo()
print(f"GPU Memory: {mem_info[0] / 1024**3:.2f} GB free / {mem_info[1] / 1024**3:.2f} GB total")

## ステップ2: 自動クリーンアップ機能を有効化

**このセルを実行するだけで、以降すべてのセル実行後に自動的にGPUメモリがクリーンアップされます。**

これで、メモリ管理を一切意識せずに計算を実行できます！

In [ ]:
# 自動クリーンアップ拡張機能をロード
import sys
sys.path.append('/workspace/colab')
%load_ext colab_auto_cleanup

## ステップ3: ライブラリのインポート

PySCFとGPU4PySCFをインポートします。

In [ ]:
import time
from pyscf import gto, dft
from gpu4pyscf import dft as gpu_dft
import numpy as np

print("✓ All libraries imported successfully")

## ステップ4: 簡単なDFT計算（水分子）

水分子のDFT計算をCPUとGPUの両方で実行します。

**注意**: セル実行終了後、自動的にGPUメモリがクリーンアップされます。

In [ ]:
# 水分子の定義
mol = gto.M(
    atom='''
    O  0.0000  0.0000  0.1173
    H  0.0000  0.7572 -0.4692
    H  0.0000 -0.7572 -0.4692
    ''',
    basis='def2-svp',
    verbose=0
)

print(f"分子: H2O")
print(f"基底関数数: {mol.nao}")
print(f"電子数: {mol.nelectron}")

In [ ]:
# CPU計算
print("CPU計算を実行中...")
start_time = time.time()

mf_cpu = dft.RKS(mol)
mf_cpu.xc = 'B3LYP'
mf_cpu.verbose = 0
energy_cpu = mf_cpu.kernel()

cpu_time = time.time() - start_time

print(f"✓ CPU計算完了: {cpu_time:.3f} 秒")
print(f"  エネルギー: {energy_cpu:.8f} Hartree")
# ← このセル終了後、自動的にメモリクリーンアップ！

In [ ]:
# GPU計算
print("GPU計算を実行中...")
print("※ 初回実行時はJITコンパイルで時間がかかる場合があります")
start_time = time.time()

mf_gpu = gpu_dft.RKS(mol)
mf_gpu.xc = 'B3LYP'
mf_gpu.verbose = 0
energy_gpu = mf_gpu.kernel()

gpu_time = time.time() - start_time

print(f"✓ GPU計算完了: {gpu_time:.3f} 秒")
print(f"  エネルギー: {energy_gpu:.8f} Hartree")
print(f"\n  CPU/GPU差分: {abs(energy_gpu - energy_cpu):.2e} Hartree")
# ← このセル終了後も自動クリーンアップ！

## ステップ5: より大きな分子（ベンゼン）

ベンゼン（C6H6）でGPU加速の効果を確認します。

In [ ]:
# ベンゼン分子の定義
mol_benzene = gto.M(
    atom='''
    C  0.0000  1.3970  0.0000
    C  1.2098  0.6985  0.0000
    C  1.2098 -0.6985  0.0000
    C  0.0000 -1.3970  0.0000
    C -1.2098 -0.6985  0.0000
    C -1.2098  0.6985  0.0000
    H  0.0000  2.4810  0.0000
    H  2.1486  1.2405  0.0000
    H  2.1486 -1.2405  0.0000
    H  0.0000 -2.4810  0.0000
    H -2.1486 -1.2405  0.0000
    H -2.1486  1.2405  0.0000
    ''',
    basis='def2-svp',
    verbose=0
)

print(f"分子: Benzene (C6H6)")
print(f"基底関数数: {mol_benzene.nao}")
print(f"電子数: {mol_benzene.nelectron}")

In [ ]:
# ベンゼンのGPU計算
print("ベンゼンのGPU計算を実行中...")
start_time = time.time()

mf_benzene = gpu_dft.RKS(mol_benzene)
mf_benzene.xc = 'B3LYP'
mf_benzene.verbose = 0
energy_benzene = mf_benzene.kernel()

benzene_time = time.time() - start_time

print(f"✓ 計算完了: {benzene_time:.3f} 秒")
print(f"  エネルギー: {energy_benzene:.8f} Hartree")
# ← 自動クリーンアップでメモリ解放！

## ステップ6: 複数回の計算（メモリ蓄積テスト）

5回連続で計算を実行し、メモリが蓄積されないことを確認します。

自動クリーンアップが有効なため、メモリリークは発生しません。

In [ ]:
# 水素分子で5回連続計算
mol_h2 = gto.M(
    atom='H 0 0 0; H 0 0 0.74',
    basis='def2-svp',
    verbose=0
)

print("5回連続計算を実行中...")
times = []
energies = []

for i in range(5):
    start = time.time()
    
    mf = gpu_dft.RKS(mol_h2)
    mf.xc = 'B3LYP'
    mf.verbose = 0
    energy = mf.kernel()
    
    elapsed = time.time() - start
    times.append(elapsed)
    energies.append(energy)
    
    print(f"  回 {i+1}/5: {elapsed:.3f} 秒, Energy: {energy:.8f} Hartree")
    # ← 各反復後に自動クリーンアップ！

print(f"\n平均時間: {np.mean(times):.3f} 秒")
print(f"エネルギーの標準偏差: {np.std(energies):.2e} Hartree")
print("✓ メモリリークなし！")

## 自動クリーンアップの管理

必要に応じて、自動クリーンアップのオン/オフを切り替えることができます。

In [ ]:
# 現在の状態を確認
%auto_cleanup_status

In [ ]:
# 詳細モード（verbose）で有効化
%auto_cleanup_on verbose
# 次のセル実行後、クリーンアップの詳細が表示されます

In [ ]:
# テスト計算（verbose modeでクリーンアップの詳細を確認）
mf = gpu_dft.RKS(mol_h2)
mf.xc = 'B3LYP'
mf.verbose = 0
energy = mf.kernel()
print(f"Energy: {energy:.8f} Hartree")
# ← セル終了後、クリーンアップの詳細が表示されます

In [ ]:
# サイレントモードに戻す
%auto_cleanup_on
# クリーンアップは実行されますが、メッセージは表示されません

## まとめ

このノートブックでは、以下を学びました：

1. **Google ColabからローカルGPU環境への接続**
2. **自動メモリクリーンアップ機能の使用**（`%load_ext colab_auto_cleanup`）
3. **GPU4PySCFでのDFT計算**
4. **メモリ管理を意識しない連続計算**

### 重要なポイント

- **`%load_ext colab_auto_cleanup`を実行するだけで、すべて自動化されます**
- 以降、どのセルを実行しても、自動的にGPUメモリがクリーンアップされます
- キャッシュやメモリ管理を一切意識する必要がありません
- 大規模な計算やループ処理でもメモリリークは発生しません

これで、RTX 5070 TiのパワーをGoogle Colabの使いやすいインターフェースから活用できます！